In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('data/autos.csv', encoding = 'ISO-8859-1')

In [ ]:
#Retirada de features que provavelmente nao afetam o preco:
data.drop(['dateCrawled','monthOfRegistration','dateCreated','nrOfPictures','lastSeen'],
          axis = 1, inplace = True)

#Como ha muitos nomes diferentes aparecendo soh uma vez, tambem eh prudente deletar:
data.drop('name', axis = 1, inplace = True)

#Retirada de dados desbalanceados:
data.drop(['seller','offerType'], axis = 1, inplace = True)
data_original = data
# %% Retirada de valores inconsistentes
# De acordo com o artigo do medium: https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba


In [ ]:
from scipy import stats
pd.set_option('display.max_columns', None)

#Lista de features numericas para analisar outliers:
numft = ['price','yearOfRegistration','powerPS','kilometer','postalCode']

#Se puramente igualasse ele consideraria o mesmo objeto (foi o que deu aqui, sei lah)
data_zscore = data.copy()

In [ ]:
# %% Criando dataframe que permite a retirada de outliers
for feat in numft:
    data_zscore[feat] = np.abs(stats.zscore(data_zscore[feat]))

In [ ]:
#Definindo threshold de 3 (3 desvios padroes):
threshold = 3
#Isso aqui vai me retornar as colunas e linhas com outliers:
iout = np.where(data_zscore[numft] > 3)

#Se eu quiser visualizar nos dados originais algum outlier a partir disso:
#Para ver o original precisaria fazer o standardscaler reverso
data.iloc[iout[0][500],iout[1][500]]

#No iout ele pode retornar duplicidade de linhas caso em uma mesma linha ocorra mais de um outlier
#Portanto este pequeno codigo se utiliza do bultin set para retornar as linhas unicas
irowsunique = list(set(list(iout[0])))

#Limpeza a partir dos indeces de linhas e colunas retornados
data_nooutlier = data.drop(data.index[irowsunique])

#Visualizacao em boxplot para verificar se realmente deletou outliers
sns.boxplot(data_nooutlier['yearOfRegistration'])


## Valores Faltantes

- Para visualizar se há valores faltantes em cada feature

In [ ]:
np.sum(data_nooutlier.isnull())/data_nooutlier.shape[0]


- Para retornar apenas as linhas que não possuem valores faltantes

In [ ]:
data_nomissing = data_nooutlier[data_nooutlier.isnull().any(axis = 1)==False]

- Não há mais valores faltantes

In [ ]:
data_nomissing.isnull().sum().sum()

- Quantas linhas retiramos em relação ao anterior

In [ ]:
data_nomissing.shape[0]/data_nooutlier.shape[0]

- Em relação a base de dados original:

In [ ]:
data_nomissing.shape[0]/data.shape[0]

# Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
def ft_str2num(data):
    #Retornar, em formato de número, quais serao as features do tipo object que serao transformadas 
    obj_features = list(data.select_dtypes(include = 'object'))
    obj_ft = [data.columns.get_loc(x) for x in obj_features]
    return obj_ft

def label_encoder(data):
    """Funcao que permite transformar todas as features do tipo object"""
    
    label_encoder = LabelEncoder()
    
    obj_ft = ft_str2num(data)
    
    #Pode ser necessario transformar para string, mas na maioria dos casos nao (afinal já é objeto)
#     def obj2str(df, features):
#     for a in features:
#         df[a] = df[a].astype(str)
#     return df

#     data = obj2str(data, object_features_train)
    
    #Funcao que realizara efetivamente a transformacao
    def le(df, obj_ft):
        le = LabelEncoder()
        for ft in obj_ft:
            df[:,ft] = le.fit_transform(df[:,ft])
        return df

    data_le = pd.DataFrame(le(data.values, obj_ft), columns = data.columns).astype(float)
    
    return data_le

In [ ]:
data_le = label_encoder(data_nomissing)

In [ ]:
data_le

# One Hot Encoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X_df = data_le.drop('price', axis = 1)

In [ ]:
X_df

In [ ]:
obj_ft = ft_str2num(data.drop('price', axis = 1))

In [ ]:
obj_ft

In [ ]:
len(obj_ft)

In [ ]:
onehot = OneHotEncoder(sparse=False, categories=obj_ft)

In [ ]:
X = X_df.values

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
ct = ColumnTransformer([('one_hot_encoder',OneHotEncoder(categories= 'auto'), obj_ft)], remainder='passthrough')

In [ ]:
X = ct.fit_transform(X).toarray()

In [ ]:
X = pd.DataFrame(X)

In [ ]:
X.head()

### Divisão entre treino e teste

In [ ]:
X = X
y = data_le['price']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

X_train.shape

# Implementação Rede Neural

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

## Arquitetura da rede neural 

- Rede neural do tipo sequencial 

In [ ]:
regressor = Sequential()

- Primeira camada oculta

In [ ]:
regressor.add(Dense(units = 150, activation ='relu', input_dim = X_train.shape[1]))

- Segunda camada oculta

In [ ]:
regressor.add(Dense(units = 150, activation ='relu'))

- Camada de saída

In [ ]:
regressor.add(Dense(units = 1, activation = 'linear'))

- Compilação da rede

In [ ]:
regressor.compile(loss = 'mean_absolute_error', 
                 optimizer = 'adam', 
                 metrics = 'mean_absolute_error')

- Resumo da rede neural 

In [ ]:
regressor.summary()

In [ ]:
regressor.fit(X_train, y_train, batch_size = 32, epochs = 100)